<a href="https://colab.research.google.com/github/micheldanjou/audio/blob/master/Voice_recording.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount google drive to save the voice samples

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!mkdir -p "./drive/My Drive/asr-new-voice-samples"

# Code for recording audio from the microphone on Google Colab
Code comes from: 
  * https://colab.research.google.com/drive/1Z6VIRZ_sX314hyev3Gm5gBqvm1wQVo-a
  * https://colab.research.google.com/gist/ricardodeazambuja/03ac98c31e87caf284f7b06286ebf7fd/microphone-to-numpy-array-from-your-browser-in-colab.ipynb
  * https://blog.addpipe.com/recording-audio-in-the-browser-using-pure-html5-and-minimal-javascript/
  * https://stackoverflow.com/a/18650249
  * https://hacks.mozilla.org/2014/06/easy-audio-capture-with-the-mediarecorder-api/
  * https://air.ghost.io/recording-to-an-audio-file-using-html5-and-js/
  * https://stackoverflow.com/a/49019356




In [ ]:
!pip install ffmpeg-python

In [ ]:
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };            
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {            
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data); 
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});
      
</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  
  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)
  
  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr

# Code for the recording campaign
Source: Michel Danjou

In [ ]:
!pip install pysoundfile

In [ ]:
import ipywidgets as widgets
from IPython.display import display
def erase_button(filename):
  button = widgets.Button(description="Delete " + filename)
  output = widgets.Output()

  def on_button_clicked(b):
    with output:
      os.remove(filename)
      print("deleted", filename)

  button.on_click(on_button_clicked)
  display(button, output)

In [ ]:
import IPython
import uuid
import scipy
import time
import scipy

recording_folder = "./recordings"

def print_sentences(s, dest, initials):
  out2 = display(IPython.display.Pretty(''), display_id=True)
  space = display(IPython.display.Pretty(''), display_id=True)
  out = display(IPython.display.Pretty(''), display_id=True)
  time.sleep(1)
  space.update(" ")
  
  wait_duration = 3
  for i in range(wait_duration, 1, -1):
    message = "WAIT " + str(i) + " sec"
    out.update(IPython.display.Pretty(message))

    out2.update(IPython.display.Pretty(s))
    time.sleep(1)

  message = "SPEAK NOW!!!!"
  out.update(IPython.display.Pretty(message))
  
  audio, sr = get_audio()
  f = initials + "___" + s.replace(" ", "_") + "___" + uuid.uuid4().hex + ".wav"
  filename = dest + "/" + f

  scipy.io.wavfile.write(filename, sr, audio)

  message = "Saved: " + filename
  out.update(IPython.display.Pretty(message))
  erase_button(filename)

# shortest list of IV (intra-venous) and PO (per oral)

amoxicillin

cefotaxime

ceftriaxone

cefuroxime

ciprofloxacin

clarithromycin

clindamycin

flucloxacillin

gentamicin

levofloxacin

meropenem

metronidazole

vancomycin

In [ ]:
sentences = []

# Sentences about patient id
sentences.append("patient id one two three four")
sentences.append("patient id five six seven eight")
sentences.append("patient id nine zero one two")
sentences.append("patient id two four six eight")
sentences.append("patient id one three five seven")

#
# amoxicillin (REVIEWED=NO)
#

# IV: https://www.medicines.org.uk/emc/product/1358/smpc
sentences.append("the patient is on day one of iv amoxicillin two grams tds")
sentences.append("the patient is on day two of iv amoxicillin two grams tds")
sentences.append("the patient is on day three of iv amoxicillin two grams tds")
sentences.append("the patient is on day four of iv amoxicillin two grams tds")
sentences.append("the patient is on day five of iv amoxicillin two grams tds")
# PO: https://www.medicines.org.uk/emc/product/11433/smpc             
sentences.append("the patient is on day one of po amoxicillin five hundred milligrams tds")
sentences.append("the patient is on day two of po amoxicillin five hundred milligrams tds")
sentences.append("the patient is on day three of po amoxicillin five hundred milligrams tds")
sentences.append("the patient is on day four of po amoxicillin five hundred milligrams tds")
sentences.append("the patient is on day five of po amoxicillin five hundred milligrams tds")


#
# cefotaxime (REVIEWED=NO)
#

# IV: https://www.medicines.org.uk/emc/product/11400/smpc
sentences.append("the patient is on day one of iv cefotaxime two grams od")
sentences.append("the patient is on day two of iv cefotaxime two grams od")
sentences.append("the patient is on day three of iv cefotaxime two grams od")
sentences.append("the patient is on day four of iv cefotaxime two grams od")
sentences.append("the patient is on day five of iv cefotaxime two grams od")

# PO: not available???

#
# ceftriaxone (REVIEWED=NO)
#

# IV: https://www.medicines.org.uk/emc/product/10604/smpc
sentences.append("the patient is on day one of iv ceftriaxone two grams od")
sentences.append("the patient is on day two of iv ceftriaxone two grams od")
sentences.append("the patient is on day three of iv ceftriaxone two grams od")
sentences.append("the patient is on day four of iv ceftriaxone two grams od")
sentences.append("the patient is on day five of iv ceftriaxone two grams od")
# PO: not available???         

#
# cefuroxime (REVIEWED=NO)
#

# IV: https://www.medicines.org.uk/emc/product/11375/smpc
sentences.append("the patient is on day one of iv cefuroxime seven hundred and fifty milligrams tds")
sentences.append("the patient is on day two of iv cefuroxime seven hundred and fifty milligrams tds")
sentences.append("the patient is on day three of iv cefuroxime seven hundred and fifty milligrams tds")
sentences.append("the patient is on day four of iv cefuroxime seven hundred and fifty milligrams tds")
sentences.append("the patient is on day five of iv cefuroxime seven hundred and fifty milligrams tds")
# PO: https://www.medicines.org.uk/emc/product/426/smpc
sentences.append("the patient is on day one of po cefuroxime two hundred and fifty milligrams bd")
sentences.append("the patient is on day two of po cefuroxime two hundred and fifty milligrams bd")
sentences.append("the patient is on day three of po cefuroxime two hundred and fifty milligrams bd")
sentences.append("the patient is on day four of po cefuroxime two hundred and fifty milligrams bd")
sentences.append("the patient is on day five of po cefuroxime two hundred and fifty milligrams bd")

#
# ciprofloxacin (REVIEWED=NO)
#

# IV: https://www.medicines.org.uk/emc/product/3796/smpc
sentences.append("the patient is on day one of iv ciprofloxacin four hundred milligrams bd")
sentences.append("the patient is on day two of iv ciprofloxacin four hundred milligrams bd")
sentences.append("the patient is on day three of iv ciprofloxacin four hundred milligrams bd")
sentences.append("the patient is on day four of iv ciprofloxacin four hundred milligrams bd")
sentences.append("the patient is on day five of iv ciprofloxacin four hundred milligrams bd")
# PO: https://www.medicines.org.uk/emc/product/9829/smpc
sentences.append("the patient is on day one of po ciprofloxacin five hundred milligrams bd")
sentences.append("the patient is on day two of po ciprofloxacin five hundred milligrams bd")
sentences.append("the patient is on day three of po ciprofloxacin five hundred milligrams bd")
sentences.append("the patient is on day four of po ciprofloxacin five hundred milligrams bd")
sentences.append("the patient is on day five of po ciprofloxacin five hundred milligrams bd")

#
# clarithromycin (REVIEWED=NO)
#

# IV: https://www.medicines.org.uk/emc/product/11347/smpc
sentences.append("the patient is on day one of iv clarithromycin five hundred milligrams bd")
sentences.append("the patient is on day two of iv clarithromycin five hundred milligrams bd")
sentences.append("the patient is on day three of iv clarithromycin five hundred milligrams bd")
sentences.append("the patient is on day four of iv clarithromycin five hundred milligrams bd")
sentences.append("the patient is on day five of iv clarithromycin five hundred milligrams bd")
# PO: https://www.medicines.org.uk/emc/product/7072/smpc
sentences.append("the patient is on day one of po clarithromycin two hundred and fifty milligrams bd")
sentences.append("the patient is on day two of po clarithromycin two hundred and fifty milligrams bd")
sentences.append("the patient is on day three of po clarithromycin two hundred and fifty milligrams bd")
sentences.append("the patient is on day four of po clarithromycin two hundred and fifty milligrams bd")
sentences.append("the patient is on day five of po clarithromycin two hundred and fifty milligrams bd")

#
# clindamycin (REVIEWED=NO)
#

# IV https://www.medicines.org.uk/emc/product/3350/smpc
sentences.append("the patient is on day one of iv clindamycin six hundred milligrams tds")
sentences.append("the patient is on day two of iv clindamycin six hundred milligrams tds")
sentences.append("the patient is on day three of iv clindamycin six hundred milligrams tds")
sentences.append("the patient is on day four of iv clindamycin six hundred milligrams tds")
sentences.append("the patient is on day five of iv clindamycin six hundred milligrams tds")
# PO: https://www.medicines.org.uk/emc/product/2485/smpc
sentences.append("the patient is on day one of po clindamycin three hundred milligrams qds")
sentences.append("the patient is on day two of po clindamycin three hundred milligrams qds")
sentences.append("the patient is on day three of po clindamycin three hundred milligrams qds")
sentences.append("the patient is on day four of po clindamycin three hundred milligrams qds")
sentences.append("the patient is on day five of po clindamycin three hundred milligrams qds")

#
# flucloxacillin (REVIEWED=YES)
#
sentences.append("the patient is on day one of iv flucloxacillin two grams qds")
sentences.append("the patient is on day two of iv flucloxacillin two grams qds")
sentences.append("the patient is on day three of iv flucloxacillin two grams qds")
sentences.append("the patient is on day four of iv flucloxacillin two grams qds")
sentences.append("the patient is on day five of iv flucloxacillin two grams qds")
             
sentences.append("the patient is on day one of po flucloxacillin five hundred milligrams qds")
sentences.append("the patient is on day two of po flucloxacillin five hundred milligrams qds")
sentences.append("the patient is on day three of po flucloxacillin five hundred milligrams qds")
sentences.append("the patient is on day four of po flucloxacillin five hundred milligrams qds")
sentences.append("the patient is on day five of po flucloxacillin five hundred milligrams qds")

#
# gentamicin (REVIEWED=NO)
#

# IV: https://www.medicines.org.uk/emc/product/2407/smpc
sentences.append("the patient is on day one of iv gentamicin four hundred milligrams od")
sentences.append("the patient is on day two of iv gentamicin four hundred milligrams od")
sentences.append("the patient is on day three of iv gentamicin four hundred milligrams od")
sentences.append("the patient is on day four of iv gentamicin four hundred milligrams od")
sentences.append("the patient is on day five of iv gentamicin four hundred milligrams od")
# PO: NO PO???


#
# levofloxacin (REVIEWED=NO)
#

# IV: https://www.medicines.org.uk/emc/product/11107/smpc
sentences.append("the patient is on day one of iv levofloxacin five hundred milligrams bd")
sentences.append("the patient is on day two of iv levofloxacin five hundred milligrams bd")
sentences.append("the patient is on day three of iv levofloxacin five hundred milligrams bd")
sentences.append("the patient is on day four of iv levofloxacin five hundred milligrams bd")
sentences.append("the patient is on day five of iv levofloxacin five hundred milligrams bd")
# PO: https://www.medicines.org.uk/emc/product/6129/smpc
sentences.append("the patient is on day one of po levofloxacin five hundred milligrams od")
sentences.append("the patient is on day two of po levofloxacin five hundred milligrams od")
sentences.append("the patient is on day three of po levofloxacin five hundred milligrams od")
sentences.append("the patient is on day four of po levofloxacin five hundred milligrams od")
sentences.append("the patient is on day five of po levofloxacin five hundred milligrams od")

#
# meropenem (REVIEWED=NO)
#
# IV: https://www.medicines.org.uk/emc/product/9834/smpc
sentences.append("the patient is on day one of iv meropenem two grams tds")
sentences.append("the patient is on day two of iv meropenem two grams tds")
sentences.append("the patient is on day three of iv meropenem two grams tds")
sentences.append("the patient is on day four of iv meropenem two grams tds")
sentences.append("the patient is on day five of iv meropenem two grams tds")
# PO:  NA??

#
# Sentences metronidazole (REVIEWED=NO)
#

# IV: https://www.medicines.org.uk/emc/product/1842/smpc
sentences.append("the patient is on day one of iv metronidazole five hundred milligrams tds")
sentences.append("the patient is on day two of iv metronidazole five hundred milligrams tds")
sentences.append("the patient is on day three of iv metronidazole five hundred milligrams tds")
sentences.append("the patient is on day four of iv metronidazole five hundred milligrams tds")
sentences.append("the patient is on day five of iv metronidazole five hundred milligrams tds")
# PO: https://www.medicines.org.uk/emc/product/10192/smpc
sentences.append("the patient is on day one of po metronidazole four hundred milligrams tds")
sentences.append("the patient is on day two of po metronidazole four hundred milligrams tds")
sentences.append("the patient is on day three of po metronidazole four hundred milligrams tds")
sentences.append("the patient is on day four of po metronidazole four hundred milligrams tds")
sentences.append("the patient is on day five of po metronidazole four hundred milligrams tds")

#
# Sentences vancomycin (REVIEWED=NO)
#

# IV: https://www.medicines.org.uk/emc/product/649/smpc
sentences.append("the patient is on day one of iv vancomycin one gram bd")
sentences.append("the patient is on day two of iv vancomycin one gram bd")
sentences.append("the patient is on day three of iv vancomycin one gram bd")
sentences.append("the patient is on day four of iv vancomycin one gram bd")
sentences.append("the patient is on day five of iv vancomycin one gram bd")
# PO: https://www.medicines.org.uk/emc/product/223/smpc
sentences.append("the patient is on day one of po vancomycin five hundred milligrams qds")
sentences.append("the patient is on day two of po vancomycin five hundred milligrams qds")
sentences.append("the patient is on day three of po vancomycin five hundred milligrams qds")
sentences.append("the patient is on day four of po vancomycin five hundred milligrams qds")
sentences.append("the patient is on day five of po vancomycin five hundred milligrams qds")

print(sentences)
print(len(sentences))

['patient id one two three four', 'patient id five six seven eight', 'patient id nine zero one two', 'patient id two four six eight', 'patient id one three five seven', 'the patient is on day one of iv amoxicillin two grams tds', 'the patient is on day two of iv amoxicillin two grams tds', 'the patient is on day three of iv amoxicillin two grams tds', 'the patient is on day four of iv amoxicillin two grams tds', 'the patient is on day five of iv amoxicillin two grams tds', 'the patient is on day one of po amoxicillin five hundred milligrams tds', 'the patient is on day two of po amoxicillin five hundred milligrams tds', 'the patient is on day three of po amoxicillin five hundred milligrams tds', 'the patient is on day four of po amoxicillin five hundred milligrams tds', 'the patient is on day five of po amoxicillin five hundred milligrams tds', 'the patient is on day one of iv cefotaxime two grams od', 'the patient is on day two of iv cefotaxime two grams od', 'the patient is on day th

# Recording campaign

In [ ]:
!rm -rf ./recordings
!mkdir -p recordings

In [ ]:
initials = input("enter your initials: ")

enter your initials: YD2


In [ ]:
print_sentences(sentences[0], recording_folder, initials)

patient id one two three four

' '

Saved: ./recordings/YD2___patient_id_one_two_three_four___1197d4c3b9794e6086923dbc5c1157b3.wav

Button(description='Delete ./recordings/YD2___patient_id_one_two_three_four___1197d4c3b9794e6086923dbc5c1157b3…

Output()

In [ ]:
print_sentences(sentences[1], recording_folder, initials)

patient id five six seven eight

' '

Saved: ./recordings/YD2___patient_id_five_six_seven_eight___8d187f9e26a44e72823cf9cc69bbc83a.wav

Button(description='Delete ./recordings/YD2___patient_id_five_six_seven_eight___8d187f9e26a44e72823cf9cc69bbc8…

Output()

In [ ]:
print_sentences(sentences[2], recording_folder, initials)

patient id nine zero one two

' '

Saved: ./recordings/YD2___patient_id_nine_zero_one_two___de8c5b3f474e4fafb67b6e6770787fc8.wav

Button(description='Delete ./recordings/YD2___patient_id_nine_zero_one_two___de8c5b3f474e4fafb67b6e6770787fc8.…

Output()

In [ ]:
print_sentences(sentences[3], recording_folder, initials)

patient id two four six eight

' '

Saved: ./recordings/YD2___patient_id_two_four_six_eight___ef1e60365d4a43bd935edbedd1ee38af.wav

Button(description='Delete ./recordings/YD2___patient_id_two_four_six_eight___ef1e60365d4a43bd935edbedd1ee38af…

Output()

In [ ]:
print_sentences(sentences[4], recording_folder, initials)

patient id one three five seven

' '

Saved: ./recordings/YD2___patient_id_one_three_five_seven___7cebc3e2a2e7446aba6f23613fcc0132.wav

Button(description='Delete ./recordings/YD2___patient_id_one_three_five_seven___7cebc3e2a2e7446aba6f23613fcc01…

Output()

In [ ]:
print_sentences(sentences[5], recording_folder, initials)

the patient is on day one of iv amoxicillin two grams tds

' '

Saved: ./recordings/YD2___the_patient_is_on_day_one_of_iv_amoxicillin_two_grams_tds___5bd83d3dde2e43ba8984c935e6181911.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_one_of_iv_amoxicillin_two_grams_tds___5bd8…

Output()

In [ ]:
print_sentences(sentences[6], recording_folder, initials)

the patient is on day two of iv amoxicillin two grams tds

' '

Saved: ./recordings/YD2___the_patient_is_on_day_two_of_iv_amoxicillin_two_grams_tds___6f851c9712754258976498d121333ebc.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_two_of_iv_amoxicillin_two_grams_tds___6f85…

Output()

In [ ]:
print_sentences(sentences[7], recording_folder, initials)

the patient is on day three of iv amoxicillin two grams tds

' '

Saved: ./recordings/YD2___the_patient_is_on_day_three_of_iv_amoxicillin_two_grams_tds___60dcfdc7416a4ebd8a363cc4afbe0ca0.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_three_of_iv_amoxicillin_two_grams_tds___60…

Output()

In [ ]:
print_sentences(sentences[8], recording_folder, initials)

the patient is on day four of iv amoxicillin two grams tds

' '

Saved: ./recordings/YD2___the_patient_is_on_day_four_of_iv_amoxicillin_two_grams_tds___e32f723a26b0472aac7796cd1ef2c8dd.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_four_of_iv_amoxicillin_two_grams_tds___e32…

Output()

In [ ]:
print_sentences(sentences[9], recording_folder, initials)

the patient is on day five of iv amoxicillin two grams tds

' '

Saved: ./recordings/YD2___the_patient_is_on_day_five_of_iv_amoxicillin_two_grams_tds___83422c5a7ec045adbfbfb844ca40e63a.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_five_of_iv_amoxicillin_two_grams_tds___834…

Output()

In [ ]:
print_sentences(sentences[10], recording_folder, initials)

the patient is on day one of po amoxicillin five hundred milligrams tds

' '

Saved: ./recordings/YD2___the_patient_is_on_day_one_of_po_amoxicillin_five_hundred_milligrams_tds___de9996c982a143a1a52a11b6cb80f3f7.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_one_of_po_amoxicillin_five_hundred_milligr…

Output()

In [ ]:
print_sentences(sentences[11], recording_folder, initials)

the patient is on day two of po amoxicillin five hundred milligrams tds

' '

Saved: ./recordings/YD2___the_patient_is_on_day_two_of_po_amoxicillin_five_hundred_milligrams_tds___3cdde296f6fa455e868119f168575271.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_two_of_po_amoxicillin_five_hundred_milligr…

Output()

In [ ]:
print_sentences(sentences[12], recording_folder, initials)

the patient is on day three of po amoxicillin five hundred milligrams tds

' '

Saved: ./recordings/YD2___the_patient_is_on_day_three_of_po_amoxicillin_five_hundred_milligrams_tds___84861a796359475da4c84c108bb76c7c.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_three_of_po_amoxicillin_five_hundred_milli…

Output()

In [ ]:
print_sentences(sentences[13], recording_folder, initials)

the patient is on day four of po amoxicillin five hundred milligrams tds

' '

Saved: ./recordings/YD2___the_patient_is_on_day_four_of_po_amoxicillin_five_hundred_milligrams_tds___066ee73816684287a7057af10638fbdc.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_four_of_po_amoxicillin_five_hundred_millig…

Output()

In [ ]:
print_sentences(sentences[14], recording_folder, initials)

the patient is on day five of po amoxicillin five hundred milligrams tds

' '

Saved: ./recordings/YD2___the_patient_is_on_day_five_of_po_amoxicillin_five_hundred_milligrams_tds___4e82b885ec6448b78bf0bc9a902f401d.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_five_of_po_amoxicillin_five_hundred_millig…

Output()

In [ ]:
print_sentences(sentences[15], recording_folder, initials)

the patient is on day one of iv cefotaxime two grams od

' '

Saved: ./recordings/YD2___the_patient_is_on_day_one_of_iv_cefotaxime_two_grams_od___aaf601c29f2e464fa3ec728f8f89aaf1.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_one_of_iv_cefotaxime_two_grams_od___aaf601…

Output()

In [ ]:
print_sentences(sentences[16], recording_folder, initials)

the patient is on day two of iv cefotaxime two grams od

' '

Saved: ./recordings/YD2___the_patient_is_on_day_two_of_iv_cefotaxime_two_grams_od___30fe58960d654556aa469c30f7d9c1c5.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_two_of_iv_cefotaxime_two_grams_od___30fe58…

Output()

In [ ]:
print_sentences(sentences[17], recording_folder, initials)

the patient is on day three of iv cefotaxime two grams od

' '

Saved: ./recordings/YD2___the_patient_is_on_day_three_of_iv_cefotaxime_two_grams_od___58b39b0882424242ab06b71861c10960.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_three_of_iv_cefotaxime_two_grams_od___58b3…

Output()

In [ ]:
print_sentences(sentences[18], recording_folder, initials)

the patient is on day four of iv cefotaxime two grams od

' '

Saved: ./recordings/YD2___the_patient_is_on_day_four_of_iv_cefotaxime_two_grams_od___0ced2aba3060435fb63fb5fb3ba2db87.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_four_of_iv_cefotaxime_two_grams_od___0ced2…

Output()

In [ ]:
print_sentences(sentences[19], recording_folder, initials)

the patient is on day five of iv cefotaxime two grams od

' '

Saved: ./recordings/YD2___the_patient_is_on_day_five_of_iv_cefotaxime_two_grams_od___a1fb1e768ce2419d9794f937d65eaa88.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_five_of_iv_cefotaxime_two_grams_od___a1fb1…

Output()

In [ ]:
print_sentences(sentences[20], recording_folder, initials)

the patient is on day one of iv ceftriaxone two grams od

' '

Saved: ./recordings/YD2___the_patient_is_on_day_one_of_iv_ceftriaxone_two_grams_od___07919b65d04541bf88905352f231c839.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_one_of_iv_ceftriaxone_two_grams_od___07919…

Output()

In [ ]:
print_sentences(sentences[21], recording_folder, initials)

the patient is on day two of iv ceftriaxone two grams od

' '

Saved: ./recordings/YD2___the_patient_is_on_day_two_of_iv_ceftriaxone_two_grams_od___b7af79f320d64ef6bfb387c351801a7c.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_two_of_iv_ceftriaxone_two_grams_od___b7af7…

Output()

In [ ]:
print_sentences(sentences[22], recording_folder, initials)

the patient is on day three of iv ceftriaxone two grams od

' '

Saved: ./recordings/YD2___the_patient_is_on_day_three_of_iv_ceftriaxone_two_grams_od___81f94c474029455a86354748896a333f.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_three_of_iv_ceftriaxone_two_grams_od___81f…

Output()

In [ ]:
print_sentences(sentences[23], recording_folder, initials)

the patient is on day four of iv ceftriaxone two grams od

' '

Saved: ./recordings/YD2___the_patient_is_on_day_four_of_iv_ceftriaxone_two_grams_od___b746740b9c9d48bfbbaeb25fb47e755a.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_four_of_iv_ceftriaxone_two_grams_od___b746…

Output()

In [ ]:
print_sentences(sentences[24], recording_folder, initials)

the patient is on day five of iv ceftriaxone two grams od

' '

Saved: ./recordings/YD2___the_patient_is_on_day_five_of_iv_ceftriaxone_two_grams_od___d45f8943169b4ae7b6e38f0cfee40594.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_five_of_iv_ceftriaxone_two_grams_od___d45f…

Output()

In [ ]:
print_sentences(sentences[25], recording_folder, initials)

the patient is on day one of iv cefuroxime seven hundred and fifty milligrams tds

' '

Saved: ./recordings/YD2___the_patient_is_on_day_one_of_iv_cefuroxime_seven_hundred_and_fifty_milligrams_tds___3b25e975d1e74de284613f87794a1910.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_one_of_iv_cefuroxime_seven_hundred_and_fif…

Output()

In [ ]:
print_sentences(sentences[26], recording_folder, initials)

the patient is on day two of iv cefuroxime seven hundred and fifty milligrams tds

' '

Saved: ./recordings/YD2___the_patient_is_on_day_two_of_iv_cefuroxime_seven_hundred_and_fifty_milligrams_tds___5c8772f421174b109499a9b2d21ecb46.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_two_of_iv_cefuroxime_seven_hundred_and_fif…

Output()

In [ ]:
print_sentences(sentences[27], recording_folder, initials)

the patient is on day three of iv cefuroxime seven hundred and fifty milligrams tds

' '

Saved: ./recordings/YD2___the_patient_is_on_day_three_of_iv_cefuroxime_seven_hundred_and_fifty_milligrams_tds___293b34048a0e4bdea2ca30083c78724b.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_three_of_iv_cefuroxime_seven_hundred_and_f…

Output()

In [ ]:
print_sentences(sentences[28], recording_folder, initials)

the patient is on day four of iv cefuroxime seven hundred and fifty milligrams tds

' '

Saved: ./recordings/YD2___the_patient_is_on_day_four_of_iv_cefuroxime_seven_hundred_and_fifty_milligrams_tds___4eaf1dcb38a640848f4c133d57e5b1e7.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_four_of_iv_cefuroxime_seven_hundred_and_fi…

Output()

In [ ]:
print_sentences(sentences[29], recording_folder, initials)

the patient is on day five of iv cefuroxime seven hundred and fifty milligrams tds

' '

Saved: ./recordings/YD2___the_patient_is_on_day_five_of_iv_cefuroxime_seven_hundred_and_fifty_milligrams_tds___f4a11bbd6a3d430ca6194c24484849cb.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_five_of_iv_cefuroxime_seven_hundred_and_fi…

Output()

In [ ]:
print_sentences(sentences[30], recording_folder, initials)

the patient is on day one of po cefuroxime two hundred and fifty milligrams bd

' '

Saved: ./recordings/YD2___the_patient_is_on_day_one_of_po_cefuroxime_two_hundred_and_fifty_milligrams_bd___29e7885826de4278bfe503143eb7bdf1.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_one_of_po_cefuroxime_two_hundred_and_fifty…

Output()

In [ ]:
print_sentences(sentences[31], recording_folder, initials)

the patient is on day two of po cefuroxime two hundred and fifty milligrams bd

' '

Saved: ./recordings/YD2___the_patient_is_on_day_two_of_po_cefuroxime_two_hundred_and_fifty_milligrams_bd___767df0eebed44c67b66a3bdfbae4b4c8.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_two_of_po_cefuroxime_two_hundred_and_fifty…

Output()

In [ ]:
print_sentences(sentences[32], recording_folder, initials)

the patient is on day three of po cefuroxime two hundred and fifty milligrams bd

' '

Saved: ./recordings/YD2___the_patient_is_on_day_three_of_po_cefuroxime_two_hundred_and_fifty_milligrams_bd___c56c4a6aae6a4af0a3213a7c1b5eda8c.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_three_of_po_cefuroxime_two_hundred_and_fif…

Output()

In [ ]:
print_sentences(sentences[33], recording_folder, initials)

the patient is on day four of po cefuroxime two hundred and fifty milligrams bd

' '

Saved: ./recordings/YD2___the_patient_is_on_day_four_of_po_cefuroxime_two_hundred_and_fifty_milligrams_bd___b17ee80e4ad443a789956c607d4d4cf7.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_four_of_po_cefuroxime_two_hundred_and_fift…

Output()

In [ ]:
print_sentences(sentences[34], recording_folder, initials)

the patient is on day five of po cefuroxime two hundred and fifty milligrams bd

' '

Saved: ./recordings/YD2___the_patient_is_on_day_five_of_po_cefuroxime_two_hundred_and_fifty_milligrams_bd___35fd5c575b974717afe2d01403263a0e.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_five_of_po_cefuroxime_two_hundred_and_fift…

Output()

In [ ]:
print_sentences(sentences[35], recording_folder, initials)

the patient is on day one of iv ciprofloxacin four hundred milligrams bd

' '

Saved: ./recordings/YD2___the_patient_is_on_day_one_of_iv_ciprofloxacin_four_hundred_milligrams_bd___80cce867afba48a2a3aa5bab28000eaf.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_one_of_iv_ciprofloxacin_four_hundred_milli…

Output()

In [ ]:
print_sentences(sentences[36], recording_folder, initials)

the patient is on day two of iv ciprofloxacin four hundred milligrams bd

' '

Saved: ./recordings/YD2___the_patient_is_on_day_two_of_iv_ciprofloxacin_four_hundred_milligrams_bd___30ca40549f9541b3acadfe3360375b8d.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_two_of_iv_ciprofloxacin_four_hundred_milli…

Output()

In [ ]:
print_sentences(sentences[37], recording_folder, initials)

the patient is on day three of iv ciprofloxacin four hundred milligrams bd

' '

Saved: ./recordings/YD2___the_patient_is_on_day_three_of_iv_ciprofloxacin_four_hundred_milligrams_bd___29e08194daf04821ba43ce1f17725a51.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_three_of_iv_ciprofloxacin_four_hundred_mil…

Output()

In [ ]:
print_sentences(sentences[38], recording_folder, initials)

the patient is on day four of iv ciprofloxacin four hundred milligrams bd

' '

Saved: ./recordings/YD2___the_patient_is_on_day_four_of_iv_ciprofloxacin_four_hundred_milligrams_bd___922880af8e5c40d5b3aa37fbfdcf1521.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_four_of_iv_ciprofloxacin_four_hundred_mill…

Output()

In [ ]:
print_sentences(sentences[39], recording_folder, initials)

the patient is on day five of iv ciprofloxacin four hundred milligrams bd

' '

Saved: ./recordings/YD2___the_patient_is_on_day_five_of_iv_ciprofloxacin_four_hundred_milligrams_bd___f896321cd2ec4f75abfb03e7fe3afa2f.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_five_of_iv_ciprofloxacin_four_hundred_mill…

Output()

In [ ]:
print_sentences(sentences[40], recording_folder, initials)

the patient is on day one of po ciprofloxacin five hundred milligrams bd

' '

Saved: ./recordings/YD2___the_patient_is_on_day_one_of_po_ciprofloxacin_five_hundred_milligrams_bd___5fe836f19768476fafb0d254c914798f.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_one_of_po_ciprofloxacin_five_hundred_milli…

Output()

In [ ]:
print_sentences(sentences[41], recording_folder, initials)

the patient is on day two of po ciprofloxacin five hundred milligrams bd

' '

Saved: ./recordings/YD2___the_patient_is_on_day_two_of_po_ciprofloxacin_five_hundred_milligrams_bd___706310e59dbd4dd599cdb66f8c8ff401.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_two_of_po_ciprofloxacin_five_hundred_milli…

Output()

In [ ]:
print_sentences(sentences[42], recording_folder, initials)

the patient is on day three of po ciprofloxacin five hundred milligrams bd

' '

Saved: ./recordings/YD2___the_patient_is_on_day_three_of_po_ciprofloxacin_five_hundred_milligrams_bd___260ded8911aa4c0b950dc898682c228b.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_three_of_po_ciprofloxacin_five_hundred_mil…

Output()

In [ ]:
print_sentences(sentences[43], recording_folder, initials)

the patient is on day four of po ciprofloxacin five hundred milligrams bd

' '

Saved: ./recordings/YD2___the_patient_is_on_day_four_of_po_ciprofloxacin_five_hundred_milligrams_bd___910f864dfe8d4045860d1d5f64ef322b.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_four_of_po_ciprofloxacin_five_hundred_mill…

Output()

In [ ]:
print_sentences(sentences[44], recording_folder, initials)

the patient is on day five of po ciprofloxacin five hundred milligrams bd

' '

Saved: ./recordings/YD2___the_patient_is_on_day_five_of_po_ciprofloxacin_five_hundred_milligrams_bd___3bddb0d4a1e44ee6aabf7d10438efa4f.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_five_of_po_ciprofloxacin_five_hundred_mill…

Output()

In [ ]:
print_sentences(sentences[45], recording_folder, initials)

the patient is on day one of iv clarithromycin five hundred milligrams bd

' '

Saved: ./recordings/YD2___the_patient_is_on_day_one_of_iv_clarithromycin_five_hundred_milligrams_bd___4debc59be60f4709ad2dcc46d3ffd8fa.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_one_of_iv_clarithromycin_five_hundred_mill…

Output()

In [ ]:
print_sentences(sentences[46], recording_folder, initials)

the patient is on day two of iv clarithromycin five hundred milligrams bd

' '

Saved: ./recordings/YD2___the_patient_is_on_day_two_of_iv_clarithromycin_five_hundred_milligrams_bd___dde87f0566044e3eb139842840df5597.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_two_of_iv_clarithromycin_five_hundred_mill…

Output()

In [ ]:
print_sentences(sentences[47], recording_folder, initials)

the patient is on day three of iv clarithromycin five hundred milligrams bd

' '

Saved: ./recordings/YD2___the_patient_is_on_day_three_of_iv_clarithromycin_five_hundred_milligrams_bd___f0a234af0dab49618c4236d9ed80b766.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_three_of_iv_clarithromycin_five_hundred_mi…

Output()

In [ ]:
print_sentences(sentences[48], recording_folder, initials)

the patient is on day four of iv clarithromycin five hundred milligrams bd

' '

Saved: ./recordings/YD2___the_patient_is_on_day_four_of_iv_clarithromycin_five_hundred_milligrams_bd___354af2bf36ce453ba1464758cc259d76.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_four_of_iv_clarithromycin_five_hundred_mil…

Output()

In [ ]:
print_sentences(sentences[49], recording_folder, initials)

the patient is on day five of iv clarithromycin five hundred milligrams bd

' '

Saved: ./recordings/YD2___the_patient_is_on_day_five_of_iv_clarithromycin_five_hundred_milligrams_bd___7f9dbfd2474b4c65b6e2c457a34f3a3e.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_five_of_iv_clarithromycin_five_hundred_mil…

Output()

In [ ]:
print_sentences(sentences[50], recording_folder, initials)

the patient is on day one of po clarithromycin two hundred and fifty milligrams bd

' '

Saved: ./recordings/YD2___the_patient_is_on_day_one_of_po_clarithromycin_two_hundred_and_fifty_milligrams_bd___3f26798bdeaa4d7f93ce0d11d01103ef.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_one_of_po_clarithromycin_two_hundred_and_f…

Output()

In [ ]:
print_sentences(sentences[51], recording_folder, initials)

the patient is on day two of po clarithromycin two hundred and fifty milligrams bd

' '

Saved: ./recordings/YD2___the_patient_is_on_day_two_of_po_clarithromycin_two_hundred_and_fifty_milligrams_bd___de99fab1f9114415b515a12bade17c41.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_two_of_po_clarithromycin_two_hundred_and_f…

Output()

In [ ]:
print_sentences(sentences[52], recording_folder, initials)

the patient is on day three of po clarithromycin two hundred and fifty milligrams bd

' '

Saved: ./recordings/YD2___the_patient_is_on_day_three_of_po_clarithromycin_two_hundred_and_fifty_milligrams_bd___c321768695c94c78ac6bb5f025a8842f.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_three_of_po_clarithromycin_two_hundred_and…

Output()

In [ ]:
print_sentences(sentences[53], recording_folder, initials)

the patient is on day four of po clarithromycin two hundred and fifty milligrams bd

' '

Saved: ./recordings/YD2___the_patient_is_on_day_four_of_po_clarithromycin_two_hundred_and_fifty_milligrams_bd___81fd4f337f054f58b4155ed3d45c1a1e.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_four_of_po_clarithromycin_two_hundred_and_…

Output()

In [ ]:
print_sentences(sentences[54], recording_folder, initials)

the patient is on day five of po clarithromycin two hundred and fifty milligrams bd

' '

Saved: ./recordings/YD2___the_patient_is_on_day_five_of_po_clarithromycin_two_hundred_and_fifty_milligrams_bd___8b092c99cbe740758cccaf36f75c1ee9.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_five_of_po_clarithromycin_two_hundred_and_…

Output()

In [ ]:
print_sentences(sentences[55], recording_folder, initials)

the patient is on day one of iv clindamycin six hundred milligrams tds

' '

Saved: ./recordings/YD2___the_patient_is_on_day_one_of_iv_clindamycin_six_hundred_milligrams_tds___daea949da2a64cd2b4626ae84caf0605.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_one_of_iv_clindamycin_six_hundred_milligra…

Output()

In [ ]:
print_sentences(sentences[56], recording_folder, initials)

the patient is on day two of iv clindamycin six hundred milligrams tds

' '

Saved: ./recordings/YD2___the_patient_is_on_day_two_of_iv_clindamycin_six_hundred_milligrams_tds___31cc881162f0438f92ce9084b458352e.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_two_of_iv_clindamycin_six_hundred_milligra…

Output()

In [ ]:
print_sentences(sentences[57], recording_folder, initials)

the patient is on day three of iv clindamycin six hundred milligrams tds

' '

Saved: ./recordings/YD2___the_patient_is_on_day_three_of_iv_clindamycin_six_hundred_milligrams_tds___adb57afcdc7e4ec89b537fab98ada650.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_three_of_iv_clindamycin_six_hundred_millig…

Output()

In [ ]:
print_sentences(sentences[58], recording_folder, initials)

the patient is on day four of iv clindamycin six hundred milligrams tds

' '

Saved: ./recordings/YD2___the_patient_is_on_day_four_of_iv_clindamycin_six_hundred_milligrams_tds___2f6fcfee44794795b801ec9242315711.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_four_of_iv_clindamycin_six_hundred_milligr…

Output()

In [ ]:
print_sentences(sentences[59], recording_folder, initials)

the patient is on day five of iv clindamycin six hundred milligrams tds

' '

Saved: ./recordings/YD2___the_patient_is_on_day_five_of_iv_clindamycin_six_hundred_milligrams_tds___0a2abf15c9b141f1bf967c61bba98981.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_five_of_iv_clindamycin_six_hundred_milligr…

Output()

In [ ]:
print_sentences(sentences[60], recording_folder, initials)

the patient is on day one of po clindamycin three hundred milligrams qds

' '

Saved: ./recordings/YD2___the_patient_is_on_day_one_of_po_clindamycin_three_hundred_milligrams_qds___9f984dd60785480887de059fb70e59e7.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_one_of_po_clindamycin_three_hundred_millig…

Output()

In [ ]:
print_sentences(sentences[61], recording_folder, initials)

the patient is on day two of po clindamycin three hundred milligrams qds

' '

Saved: ./recordings/YD2___the_patient_is_on_day_two_of_po_clindamycin_three_hundred_milligrams_qds___ea00fd1571214103b4cc62b4ff68f239.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_two_of_po_clindamycin_three_hundred_millig…

Output()

In [ ]:
print_sentences(sentences[62], recording_folder, initials)

the patient is on day three of po clindamycin three hundred milligrams qds

' '

Saved: ./recordings/YD2___the_patient_is_on_day_three_of_po_clindamycin_three_hundred_milligrams_qds___7732add9c2484a6281c1cbbd3fc8eedc.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_three_of_po_clindamycin_three_hundred_mill…

Output()

In [ ]:
print_sentences(sentences[63], recording_folder, initials)

the patient is on day four of po clindamycin three hundred milligrams qds

' '

Saved: ./recordings/YD2___the_patient_is_on_day_four_of_po_clindamycin_three_hundred_milligrams_qds___d0b7c90774384084a318f2e08cde973c.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_four_of_po_clindamycin_three_hundred_milli…

Output()

In [ ]:
print_sentences(sentences[64], recording_folder, initials)

the patient is on day five of po clindamycin three hundred milligrams qds

' '

Saved: ./recordings/YD2___the_patient_is_on_day_five_of_po_clindamycin_three_hundred_milligrams_qds___e568cae113b140d596dbe6d96e6a03b0.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_five_of_po_clindamycin_three_hundred_milli…

Output()

In [ ]:
print_sentences(sentences[65], recording_folder, initials)

the patient is on day one of iv flucloxacillin two grams qds

' '

Saved: ./recordings/YD2___the_patient_is_on_day_one_of_iv_flucloxacillin_two_grams_qds___dc98bc2bc20c4722a217471fadd2669b.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_one_of_iv_flucloxacillin_two_grams_qds___d…

Output()

In [ ]:
print_sentences(sentences[66], recording_folder, initials)

the patient is on day two of iv flucloxacillin two grams qds

' '

Saved: ./recordings/YD2___the_patient_is_on_day_two_of_iv_flucloxacillin_two_grams_qds___d4d897e488fe4e8090d20f91e5f804a2.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_two_of_iv_flucloxacillin_two_grams_qds___d…

Output()

In [ ]:
print_sentences(sentences[67], recording_folder, initials)

the patient is on day three of iv flucloxacillin two grams qds

' '

Saved: ./recordings/YD2___the_patient_is_on_day_three_of_iv_flucloxacillin_two_grams_qds___a76c8cc7b818401c8eb28517e61b6cb7.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_three_of_iv_flucloxacillin_two_grams_qds__…

Output()

In [ ]:
print_sentences(sentences[68], recording_folder, initials)

the patient is on day four of iv flucloxacillin two grams qds

' '

Saved: ./recordings/YD2___the_patient_is_on_day_four_of_iv_flucloxacillin_two_grams_qds___be8c4a24be7b4bf8b444cb9165ff2548.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_four_of_iv_flucloxacillin_two_grams_qds___…

Output()

In [ ]:
print_sentences(sentences[69], recording_folder, initials)

the patient is on day five of iv flucloxacillin two grams qds

' '

Saved: ./recordings/YD2___the_patient_is_on_day_five_of_iv_flucloxacillin_two_grams_qds___d704ef0d924b4241aadaddc41b919da9.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_five_of_iv_flucloxacillin_two_grams_qds___…

Output()

In [ ]:
print_sentences(sentences[70], recording_folder, initials)

the patient is on day one of po flucloxacillin five hundred milligrams qds

' '

Saved: ./recordings/YD2___the_patient_is_on_day_one_of_po_flucloxacillin_five_hundred_milligrams_qds___c9a9d5ac9a094defa98288bb2893c1fc.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_one_of_po_flucloxacillin_five_hundred_mill…

Output()

In [ ]:
print_sentences(sentences[71], recording_folder, initials)

the patient is on day two of po flucloxacillin five hundred milligrams qds

' '

Saved: ./recordings/YD2___the_patient_is_on_day_two_of_po_flucloxacillin_five_hundred_milligrams_qds___1181929091ab4ba4a8bc377e7c0e5750.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_two_of_po_flucloxacillin_five_hundred_mill…

Output()

In [ ]:
print_sentences(sentences[72], recording_folder, initials)

the patient is on day three of po flucloxacillin five hundred milligrams qds

' '

Saved: ./recordings/YD2___the_patient_is_on_day_three_of_po_flucloxacillin_five_hundred_milligrams_qds___9f86928ca6bb4445bb31b1c05bc18e12.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_three_of_po_flucloxacillin_five_hundred_mi…

Output()

In [ ]:
print_sentences(sentences[73], recording_folder, initials)

the patient is on day four of po flucloxacillin five hundred milligrams qds

' '

Saved: ./recordings/YD2___the_patient_is_on_day_four_of_po_flucloxacillin_five_hundred_milligrams_qds___07f58f6bd8cc4b33bab5a95da7f94391.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_four_of_po_flucloxacillin_five_hundred_mil…

Output()

In [ ]:
print_sentences(sentences[74], recording_folder, initials)

the patient is on day five of po flucloxacillin five hundred milligrams qds

' '

Saved: ./recordings/YD2___the_patient_is_on_day_five_of_po_flucloxacillin_five_hundred_milligrams_qds___0ae85a609f0d419fb2459ee85575dd6b.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_five_of_po_flucloxacillin_five_hundred_mil…

Output()

In [ ]:
print_sentences(sentences[75], recording_folder, initials)

the patient is on day one of iv gentamicin four hundred milligrams od

' '

Saved: ./recordings/YD2___the_patient_is_on_day_one_of_iv_gentamicin_four_hundred_milligrams_od___b2d84288dd224d9a97f19a78cb9b9a03.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_one_of_iv_gentamicin_four_hundred_milligra…

Output()

In [ ]:
print_sentences(sentences[76], recording_folder, initials)

the patient is on day two of iv gentamicin four hundred milligrams od

' '

Saved: ./recordings/YD2___the_patient_is_on_day_two_of_iv_gentamicin_four_hundred_milligrams_od___7015f61f2b134f1dba86e470df56161c.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_two_of_iv_gentamicin_four_hundred_milligra…

Output()

In [ ]:
print_sentences(sentences[77], recording_folder, initials)

the patient is on day three of iv gentamicin four hundred milligrams od

' '

Saved: ./recordings/YD2___the_patient_is_on_day_three_of_iv_gentamicin_four_hundred_milligrams_od___346d8719db234ad985e47482bd9fbc22.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_three_of_iv_gentamicin_four_hundred_millig…

Output()

In [ ]:
print_sentences(sentences[78], recording_folder, initials)

the patient is on day four of iv gentamicin four hundred milligrams od

' '

Saved: ./recordings/YD2___the_patient_is_on_day_four_of_iv_gentamicin_four_hundred_milligrams_od___f0650d529029462d9272f303953a6c79.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_four_of_iv_gentamicin_four_hundred_milligr…

Output()

In [ ]:
print_sentences(sentences[79], recording_folder, initials)

the patient is on day five of iv gentamicin four hundred milligrams od

' '

Saved: ./recordings/YD2___the_patient_is_on_day_five_of_iv_gentamicin_four_hundred_milligrams_od___73a423f39fd44bd3b9f28321971034a4.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_five_of_iv_gentamicin_four_hundred_milligr…

Output()

In [ ]:
print_sentences(sentences[80], recording_folder, initials)

the patient is on day one of iv levofloxacin five hundred milligrams bd

' '

Saved: ./recordings/YD2___the_patient_is_on_day_one_of_iv_levofloxacin_five_hundred_milligrams_bd___9d1b94f4de524f398812b5bf059e2310.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_one_of_iv_levofloxacin_five_hundred_millig…

Output()

In [ ]:
print_sentences(sentences[81], recording_folder, initials)

the patient is on day two of iv levofloxacin five hundred milligrams bd

' '

Saved: ./recordings/YD2___the_patient_is_on_day_two_of_iv_levofloxacin_five_hundred_milligrams_bd___c2222590d4874fffa7b618118992ee52.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_two_of_iv_levofloxacin_five_hundred_millig…

Output()

In [ ]:
print_sentences(sentences[82], recording_folder, initials)

the patient is on day three of iv levofloxacin five hundred milligrams bd

' '

Saved: ./recordings/YD2___the_patient_is_on_day_three_of_iv_levofloxacin_five_hundred_milligrams_bd___032598d67d564c12949cbd0620880408.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_three_of_iv_levofloxacin_five_hundred_mill…

Output()

In [ ]:
print_sentences(sentences[83], recording_folder, initials)

the patient is on day four of iv levofloxacin five hundred milligrams bd

' '

Saved: ./recordings/YD2___the_patient_is_on_day_four_of_iv_levofloxacin_five_hundred_milligrams_bd___e7f8cffb2fbf4a7bad46c815497f3cd9.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_four_of_iv_levofloxacin_five_hundred_milli…

Output()

In [ ]:
print_sentences(sentences[84], recording_folder, initials)

the patient is on day five of iv levofloxacin five hundred milligrams bd

' '

Saved: ./recordings/YD2___the_patient_is_on_day_five_of_iv_levofloxacin_five_hundred_milligrams_bd___1277dbe987db4666b997ef3a9439fcd6.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_five_of_iv_levofloxacin_five_hundred_milli…

Output()

In [ ]:
print_sentences(sentences[85], recording_folder, initials)

the patient is on day one of po levofloxacin five hundred milligrams od

' '

Saved: ./recordings/YD2___the_patient_is_on_day_one_of_po_levofloxacin_five_hundred_milligrams_od___0842359eeab84305a00d2b312e750cb8.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_one_of_po_levofloxacin_five_hundred_millig…

Output()

In [ ]:
print_sentences(sentences[86], recording_folder, initials)

the patient is on day two of po levofloxacin five hundred milligrams od

' '

Saved: ./recordings/YD2___the_patient_is_on_day_two_of_po_levofloxacin_five_hundred_milligrams_od___be6c45944fcf4328aed6daf1c6469983.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_two_of_po_levofloxacin_five_hundred_millig…

Output()

In [ ]:
print_sentences(sentences[87], recording_folder, initials)

the patient is on day three of po levofloxacin five hundred milligrams od

' '

Saved: ./recordings/YD2___the_patient_is_on_day_three_of_po_levofloxacin_five_hundred_milligrams_od___f25a392081bd4eabad8b94972704d0f9.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_three_of_po_levofloxacin_five_hundred_mill…

Output()

In [ ]:
print_sentences(sentences[88], recording_folder, initials)

the patient is on day four of po levofloxacin five hundred milligrams od

' '

Saved: ./recordings/YD2___the_patient_is_on_day_four_of_po_levofloxacin_five_hundred_milligrams_od___194282ead76b4955b8ec01fbfd6a35a4.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_four_of_po_levofloxacin_five_hundred_milli…

Output()

In [ ]:
print_sentences(sentences[89], recording_folder, initials)

the patient is on day five of po levofloxacin five hundred milligrams od

' '

Saved: ./recordings/YD2___the_patient_is_on_day_five_of_po_levofloxacin_five_hundred_milligrams_od___67cd5153e0584f41bef4ae60eb82c25a.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_five_of_po_levofloxacin_five_hundred_milli…

Output()

In [ ]:
print_sentences(sentences[90], recording_folder, initials)

the patient is on day one of iv meropenem two grams tds

' '

Saved: ./recordings/YD2___the_patient_is_on_day_one_of_iv_meropenem_two_grams_tds___808dfdc3089940bd9b405f65c113b4f2.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_one_of_iv_meropenem_two_grams_tds___808dfd…

Output()

In [ ]:
print_sentences(sentences[91], recording_folder, initials)

the patient is on day two of iv meropenem two grams tds

' '

Saved: ./recordings/YD2___the_patient_is_on_day_two_of_iv_meropenem_two_grams_tds___5a3854507529486da76f2795a3fbcc2c.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_two_of_iv_meropenem_two_grams_tds___5a3854…

Output()

In [ ]:
print_sentences(sentences[92], recording_folder, initials)

the patient is on day three of iv meropenem two grams tds

' '

Saved: ./recordings/YD2___the_patient_is_on_day_three_of_iv_meropenem_two_grams_tds___e84c7289a3634f678fc57b471795aed7.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_three_of_iv_meropenem_two_grams_tds___e84c…

Output()

In [ ]:
print_sentences(sentences[93], recording_folder, initials)

the patient is on day four of iv meropenem two grams tds

' '

Saved: ./recordings/YD2___the_patient_is_on_day_four_of_iv_meropenem_two_grams_tds___0421e20e17074ee38e844beb38aa4535.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_four_of_iv_meropenem_two_grams_tds___0421e…

Output()

In [ ]:
print_sentences(sentences[94], recording_folder, initials)

the patient is on day five of iv meropenem two grams tds

' '

Saved: ./recordings/YD2___the_patient_is_on_day_five_of_iv_meropenem_two_grams_tds___c7b4bbc7758f4c208b1a728637ee5ab0.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_five_of_iv_meropenem_two_grams_tds___c7b4b…

Output()

In [ ]:
print_sentences(sentences[95], recording_folder, initials)

the patient is on day one of iv metronidazole five hundred milligrams tds

' '

Saved: ./recordings/YD2___the_patient_is_on_day_one_of_iv_metronidazole_five_hundred_milligrams_tds___edc5123579ff4d26bcb666d322c085fd.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_one_of_iv_metronidazole_five_hundred_milli…

Output()

In [ ]:
print_sentences(sentences[96], recording_folder, initials)

the patient is on day two of iv metronidazole five hundred milligrams tds

' '

Saved: ./recordings/YD2___the_patient_is_on_day_two_of_iv_metronidazole_five_hundred_milligrams_tds___221a7ba2d80947b4a8f9ca0bbad15a7b.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_two_of_iv_metronidazole_five_hundred_milli…

Output()

In [ ]:
print_sentences(sentences[97], recording_folder, initials)

the patient is on day three of iv metronidazole five hundred milligrams tds

' '

Saved: ./recordings/YD2___the_patient_is_on_day_three_of_iv_metronidazole_five_hundred_milligrams_tds___0a1738716c8d4b1691def4b3ff0afce8.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_three_of_iv_metronidazole_five_hundred_mil…

Output()

In [ ]:
print_sentences(sentences[98], recording_folder, initials)

the patient is on day four of iv metronidazole five hundred milligrams tds

' '

Saved: ./recordings/YD2___the_patient_is_on_day_four_of_iv_metronidazole_five_hundred_milligrams_tds___824ede0613b54f999e90ae95149b0fda.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_four_of_iv_metronidazole_five_hundred_mill…

Output()

In [ ]:
print_sentences(sentences[99], recording_folder, initials)

the patient is on day five of iv metronidazole five hundred milligrams tds

' '

Saved: ./recordings/YD2___the_patient_is_on_day_five_of_iv_metronidazole_five_hundred_milligrams_tds___ddd8b5e670994381a7f81236821c98d2.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_five_of_iv_metronidazole_five_hundred_mill…

Output()

In [ ]:
print_sentences(sentences[100], recording_folder, initials)

the patient is on day one of po metronidazole four hundred milligrams tds

' '

Saved: ./recordings/YD2___the_patient_is_on_day_one_of_po_metronidazole_four_hundred_milligrams_tds___e98c73ff5402436f9a50f871cb043d30.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_one_of_po_metronidazole_four_hundred_milli…

Output()

In [ ]:
print_sentences(sentences[101], recording_folder, initials)

the patient is on day two of po metronidazole four hundred milligrams tds

' '

Saved: ./recordings/YD2___the_patient_is_on_day_two_of_po_metronidazole_four_hundred_milligrams_tds___574ea3b96aa24e1697144228d172d71c.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_two_of_po_metronidazole_four_hundred_milli…

Output()

In [ ]:
print_sentences(sentences[102], recording_folder, initials)

the patient is on day three of po metronidazole four hundred milligrams tds

' '

Saved: ./recordings/YD2___the_patient_is_on_day_three_of_po_metronidazole_four_hundred_milligrams_tds___02598cb1dec5446c9ef2075a08f4d614.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_three_of_po_metronidazole_four_hundred_mil…

Output()

In [ ]:
print_sentences(sentences[103], recording_folder, initials)

the patient is on day four of po metronidazole four hundred milligrams tds

' '

Saved: ./recordings/YD2___the_patient_is_on_day_four_of_po_metronidazole_four_hundred_milligrams_tds___f1253bdf96274a44b7302ae784c81cce.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_four_of_po_metronidazole_four_hundred_mill…

Output()

In [ ]:
print_sentences(sentences[104], recording_folder, initials)

the patient is on day five of po metronidazole four hundred milligrams tds

' '

Saved: ./recordings/YD2___the_patient_is_on_day_five_of_po_metronidazole_four_hundred_milligrams_tds___557a5e7115e247fb8752f8a2c75f6cf8.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_five_of_po_metronidazole_four_hundred_mill…

Output()

In [ ]:
print_sentences(sentences[105], recording_folder, initials)

the patient is on day one of iv vancomycin one gram bd

' '

Saved: ./recordings/YD2___the_patient_is_on_day_one_of_iv_vancomycin_one_gram_bd___6d03fbecfef54a6ca3887627e71e9ace.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_one_of_iv_vancomycin_one_gram_bd___6d03fbe…

Output()

In [ ]:
print_sentences(sentences[106], recording_folder, initials)

the patient is on day two of iv vancomycin one gram bd

' '

Saved: ./recordings/YD2___the_patient_is_on_day_two_of_iv_vancomycin_one_gram_bd___a9656615f6ec4bb4b581ae09e044fca2.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_two_of_iv_vancomycin_one_gram_bd___a965661…

Output()

In [ ]:
print_sentences(sentences[107], recording_folder, initials)

the patient is on day three of iv vancomycin one gram bd

' '

Saved: ./recordings/YD2___the_patient_is_on_day_three_of_iv_vancomycin_one_gram_bd___0c64e31b2b98440687dbb5bb84a6b6d5.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_three_of_iv_vancomycin_one_gram_bd___0c64e…

Output()

In [ ]:
print_sentences(sentences[108], recording_folder, initials)

the patient is on day four of iv vancomycin one gram bd

' '

Saved: ./recordings/YD2___the_patient_is_on_day_four_of_iv_vancomycin_one_gram_bd___aeee92ce52bf4148b0e34f4f8cc46816.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_four_of_iv_vancomycin_one_gram_bd___aeee92…

Output()

In [ ]:
print_sentences(sentences[109], recording_folder, initials)

the patient is on day five of iv vancomycin one gram bd

' '

Saved: ./recordings/YD2___the_patient_is_on_day_five_of_iv_vancomycin_one_gram_bd___daa71598c1384c27a4833f2a3e4604aa.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_five_of_iv_vancomycin_one_gram_bd___daa715…

Output()

In [ ]:
print_sentences(sentences[110], recording_folder, initials)

the patient is on day one of po vancomycin five hundred milligrams qds

' '

Saved: ./recordings/YD2___the_patient_is_on_day_one_of_po_vancomycin_five_hundred_milligrams_qds___9ca33aa99f79442fbd6d9b770332f076.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_one_of_po_vancomycin_five_hundred_milligra…

Output()

In [ ]:
print_sentences(sentences[111], recording_folder, initials)

the patient is on day two of po vancomycin five hundred milligrams qds

' '

Saved: ./recordings/YD2___the_patient_is_on_day_two_of_po_vancomycin_five_hundred_milligrams_qds___7ba83308d60b40e3b2db68ae350ee472.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_two_of_po_vancomycin_five_hundred_milligra…

Output()

In [ ]:
print_sentences(sentences[112], recording_folder, initials)

the patient is on day three of po vancomycin five hundred milligrams qds

' '

Saved: ./recordings/YD2___the_patient_is_on_day_three_of_po_vancomycin_five_hundred_milligrams_qds___0ab00c8f7e1e47bea7f703d74a90bb82.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_three_of_po_vancomycin_five_hundred_millig…

Output()

In [ ]:
print_sentences(sentences[113], recording_folder, initials)

the patient is on day four of po vancomycin five hundred milligrams qds

' '

Saved: ./recordings/YD2___the_patient_is_on_day_four_of_po_vancomycin_five_hundred_milligrams_qds___bb37a49de530457996e805db1ca7994a.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_four_of_po_vancomycin_five_hundred_milligr…

Output()

In [ ]:
print_sentences(sentences[114], recording_folder, initials)

the patient is on day five of po vancomycin five hundred milligrams qds

' '

Saved: ./recordings/YD2___the_patient_is_on_day_five_of_po_vancomycin_five_hundred_milligrams_qds___5ba721d0e124435d85001bc8188ead4c.wav

Button(description='Delete ./recordings/YD2___the_patient_is_on_day_five_of_po_vancomycin_five_hundred_milligr…

Output()

# Save the voice samples on google drive

In [ ]:
!cp -r ./recordings/* ./drive/My\ Drive/asr-new-voice-samples

# 
Generate NeMo manifest

In [ ]:
import os
entries = os.listdir(recording_folder)
for entry in entries:
  if ".wav" in entry:
    print(entry)
    transcript_underscore = file.split("__",1)[0] 
    



In [ ]:
!mkdir -p ./recordings/train/
!mkdir -p ./recordings/test/
!mkdir -p ./recordings/dev/

In [ ]:
nemo_train_folder = './recordings/train/'
nemo_test_folder = './recordings/test/'
nemo_dev_folder = './recordings/dev/'

In [ ]:
X_train, X_test, y_train, y_test 
    = train_test_split(X, y, test_size=0.2, random_state=1)

 X_train, X_val, y_train, y_val 
    = train_test_split(X_train, y_train, test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2
